In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
from anndata import AnnData
from scipy.sparse import issparse
from scipy.sparse import csr_array
import anndata as ad

In [2]:
adata_exp0 = ad.read_h5ad('C:/Users/mdichgan/Documents/Helmholtz/send_to_Jakob/spatial/counts_CPc_exp0_BA28.h5ad')
adata_Yao = ad.read_h5ad(
    'C:/Users/mdichgan/Documents/Helmholtz/send_to_Jakob/sc/Yao_150kcells_subsample_with_annotations_sparse_subset.h5ad')

In [3]:
adata_Yao.obs["celltype"] = adata_Yao.obs["label"]
adata_exp0.layers["raw"] = adata_exp0.X
adata_Yao.layers["raw"] = adata_Yao.X

# sc.pp.normalize_total(adata_exp0)
# sc.pp.normalize_total(adata_Yao)
adata_exp0.layers["lognorm"] = adata_exp0.X
adata_Yao.layers["lognorm"] = adata_Yao.X

In [4]:
#adata_sp has no x,y coordinates, temporary random:
adata_exp0.obs["x"] = np.random.uniform(0,200,23282)
adata_exp0.obs["y"] = np.random.uniform(0,100,23282)

In [128]:
def relative_pairwise_celltype_expression_local(adata_sp: AnnData, adata_sc: AnnData, bins_x: list[float], bins_y: list[float], key:str='celltype', layer:str='lognorm'):

    min_number_cells = 1
    n_bins_x = len(bins_x) - 1
    n_bins_y = len(bins_y) - 1

    ### SET UP
    # set the .X layer of each of the adatas to be log-normalized counts
    adata_sp.X = adata_sp.layers[layer]
    adata_sc.X = adata_sc.layers[layer]

    # take the intersection of genes in adata_sp and adata_sc, as a list
    intersect_genes = list(set(adata_sp.var_names).intersection(set(adata_sc.var_names)))
    n_intersect_genes = len(intersect_genes)

    # subset adata_sc and adata_sp to only include genes in the intersection of adata_sp and adata_sc 
    adata_sc=adata_sc[:,intersect_genes]
    adata_sp=adata_sp[:,intersect_genes]
            
    # find the intersection of unique celltypes in adata_sc and adata_sp
    intersect_celltypes = list(set(adata_sp.obs["celltype"]).intersection(set(adata_sc.obs["celltype"])))

    adata_sc = adata_sc[adata_sc.obs[key].isin(intersect_celltypes), :]
    adata_sp = adata_sp[adata_sp.obs[key].isin(intersect_celltypes), :]

    # for easier subsetting later
    adata_sp.obs.index = adata_sp.obs.index.str[5:].astype(int)

    # get expression arrays and sum of expression for normalization
    sc_X = adata_sc.X.toarray()
    sp_X = adata_sp.X.toarray()
    sum_sc = np.sum(sc_X)

    # get the adata_sc cell x gene matrix as a pandas dataframe (w gene names as column names)
    exp_sc=pd.DataFrame(sc_X,columns=adata_sc.var.index)        

    # get the adata_sp cell x gene matrix as a pandas dataframe (w gene names as column names)
    exp_sp=pd.DataFrame(sp_X,columns=adata_sp.var.index)

    # add "celltype" label column to exp_sc & exp_sp cell x gene matrices 
    exp_sc[key]=list(adata_sc.obs[key])
    exp_sp[key]=list(adata_sp.obs[key])

    exp_sp.index = adata_sp.obs.index

    # get the mean expression per celltype and normalize by sum_sc and n_intersect_genes 
    if sum_sc != 0:
        mean_celltype_sc_normalized=(exp_sc.groupby(key).mean()/sum_sc)*n_intersect_genes
    else: 
        mean_celltype_sc_normalized=0

    # get pairwise differences of normalized mean expression of sc
    mean_celltype_sc_normalized = mean_celltype_sc_normalized.to_numpy()
    pairwise_diff_sc = mean_celltype_sc_normalized[:,:,np.newaxis] - mean_celltype_sc_normalized[:,np.newaxis,:]

    gridfield_metric = np.zeros((n_bins_y,n_bins_x))

    i, j = 0, 0
    for x_start, x_end in zip(bins_x[:-1], bins_x[1:]):
        i = 0
        for y_start, y_end in zip(bins_y[:-1], bins_y[1:]):    
            #hat adata_sp.obs immer x, y Koord? Sonst mit get_cells_location Funktion
            df = adata_sp.obs[["x", "y"]]
            df = df[
            (df["x"] >= x_start)
            & (df["x"] < x_end)
            & (df["y"] >= y_start)
            & (df["y"] < y_end)
        ]

            sp_local = exp_sp.loc[df.index,:]          
            sum_sp_local = np.sum(np.sum(sp_local.iloc[:,:-1]))

            if (any(sp_local[key].value_counts() < 1)) | (not set(list(exp_sc[key].unique())).issubset(set(list(sp_local[key].unique())))):
                gridfield_metric[n_bins_y-1-i,j] = np.nan
                i += 1
                continue        
                
            # find the mean expression for each gene for each celltype in sc and sp data
            mean_celltype_sp_normalized=sp_local.groupby(key).mean()/sum_sp_local
            mean_celltype_sp_normalized=mean_celltype_sp_normalized.to_numpy()
            pairwise_diff_sp = mean_celltype_sp_normalized[:,:,np.newaxis] - mean_celltype_sp_normalized[:,np.newaxis,:]

            delta = np.sum(np.abs(pairwise_diff_sp-pairwise_diff_sc))
            gridfield_metric[n_bins_y-1-i,j]  = 1-delta/(2*np.sum(np.abs(pairwise_diff_sc)))
            i+=1
        j+=1 

    return gridfield_metric
        

In [131]:
relative_pairwise_celltype_expression_local(adata_exp0,adata_Yao,list(range(0,201,40)),list(range(0,101,50)))

array([[0.51178149, 0.51119454, 0.51139832, 0.51090119, 0.51050044],
       [0.51131005, 0.51052719, 0.51128593, 0.51093208, 0.51183216]])

In [38]:
layer = "lognorm"
key = "celltype"
adata_sp, adata_sc = adata_exp0, adata_Yao
bins_x, bins_y = list(range(0,201,20)),list(range(1,101,20))
min_number_cells = 10 
n_bins_x = len(bins_x) - 1
n_bins_y = len(bins_y) - 1

### SET UP
# set the .X layer of each of the adatas to be log-normalized counts
adata_sp.X = adata_sp.layers[layer]
adata_sc.X = adata_sc.layers[layer]

# take the intersection of genes in adata_sp and adata_sc, as a list
intersect_genes = list(set(adata_sp.var_names).intersection(set(adata_sc.var_names)))
n_intersect_genes = len(intersect_genes)

# subset adata_sc and adata_sp to only include genes in the intersection of adata_sp and adata_sc 
adata_sc=adata_sc[:,intersect_genes]
adata_sp=adata_sp[:,intersect_genes]
        
# find the intersection of unique celltypes in adata_sc and adata_sp
intersect_celltypes = list(set(adata_sp.obs["celltype"]).intersection(set(adata_sc.obs["celltype"])))

adata_sc = adata_sc[adata_sc.obs[key].isin(intersect_celltypes), :]
adata_sp = adata_sp[adata_sp.obs[key].isin(intersect_celltypes), :]

# for easier subsetting later
adata_sp.obs.index = adata_sp.obs.index.str[5:].astype(int)

# get expression arrays and sum of expression for normalization
sc_X = adata_sc.X.toarray()
sp_X = adata_sp.X.toarray()
sum_sc = np.sum(sc_X)

# get the adata_sc cell x gene matrix as a pandas dataframe (w gene names as column names)
exp_sc=pd.DataFrame(sc_X,columns=adata_sc.var.index)        

# get the adata_sp cell x gene matrix as a pandas dataframe (w gene names as column names)
exp_sp=pd.DataFrame(sp_X,columns=adata_sp.var.index)

# add "celltype" label column to exp_sc & exp_sp cell x gene matrices 
exp_sc[key]=list(adata_sc.obs[key])
exp_sp[key]=list(adata_sp.obs[key])

exp_sp.index = adata_sp.obs.index

In [42]:
# get the mean expression per celltype and normalize by sum_sc and n_intersect_genes 
if sum_sc != 0:
    mean_celltype_sc_normalized=(exp_sc.groupby(key).mean()/sum_sc)*n_intersect_genes
else: 
    mean_celltype_sc_normalized=0

In [ ]:
# get pairwise differences of normalized mean expression of sc
mean_celltype_sc_normalized = mean_celltype_sc_normalized.to_numpy()
pairwise_diff_sc = mean_celltype_sc_normalized[:,:,np.newaxis] - mean_celltype_sc_normalized[:,np.newaxis,:]

In [85]:
gridfield_metric = np.zeros((n_bins_y,n_bins_x))
np.shape(gridfield_metric)

(4, 10)

In [125]:
i, j = 0, 0
for x_start, x_end in zip(bins_x[:-1], bins_x[1:]):
    i = 0
    for y_start, y_end in zip(bins_y[:-1], bins_y[1:]):    
        #hat adata_sp.obs immer x, y Koord? Sonst mit get_cells_location Funktion
        df = adata_sp.obs[["x", "y"]]
        df = df[
          (df["x"] >= x_start)
        & (df["x"] < x_end)
        & (df["y"] >= y_start)
        & (df["y"] < y_end)
    ]

        sp_local = exp_sp.loc[df.index,:]          
        sum_sp_local = np.sum(np.sum(sp_local.iloc[:,:-1]))

        if (any(sp_local[key].value_counts() < 1)) | (not set(list(exp_sc[key].unique())).issubset(set(list(sp_local[key].unique())))):
            gridfield_metric[n_bins_y-1-i,j] = np.nan
            i += 1
            continue        
            
        # find the mean expression for each gene for each celltype in sc and sp data
        mean_celltype_sp_normalized=sp_local.groupby(key).mean()/sum_sp_local
        mean_celltype_sp_normalized=mean_celltype_sp_normalized.to_numpy()
        pairwise_diff_sp = mean_celltype_sp_normalized[:,:,np.newaxis] - mean_celltype_sp_normalized[:,np.newaxis,:]

        delta = np.sum(np.abs(pairwise_diff_sp-pairwise_diff_sc))
        gridfield_metric[n_bins_y-1-i,j]  = 1-delta/(2*np.sum(np.abs(pairwise_diff_sc)))
        i+=1
    j+=1 


In [126]:
gridfield_metric

array([[0.5507064 ,        nan,        nan,        nan,        nan,
        0.55387133, 0.54822223,        nan, 0.54959879, 0.54201472],
       [       nan, 0.55403018,        nan,        nan, 0.55726358,
               nan, 0.54124294,        nan,        nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan],
       [       nan,        nan,        nan, 0.54214209,        nan,
               nan,        nan,        nan, 0.55475839,        nan]])

In [101]:
sp_local["celltype"].unique()

array(['CA3', 'Car3', 'Lamp5', 'Oligo', 'L6b CTX', 'L6 CT CTX', 'Vip',
       'Pvalb', 'CA1-ProS', 'Astro', 'Endo', 'L4/5 IT CTX', 'L5 IT CTX',
       'Sst', 'SMC-Peri', 'L5 PT CTX', 'Micro-PVM', 'VLMC', 'DG',
       'L5 NP CTX', 'CR'], dtype=object)

In [110]:
set(list(exp_sc["celltype"].unique()))

{'Meis2'}

In [121]:
set(list(exp_sc["celltype"].unique())).issubset(set(list(sp_local["celltype"].unique())))

False

In [ ]:
a = np.array([[1,2],[0,2],[0,3]])
a[:,:,np.newaxis] - a[:,np.newaxis,:]


In [ ]:
adata = ad.AnnData(np.array([[0,1],[1,2],[5,4],[0,2]]))
adata

In [ ]:
#test

t = pd.DataFrame(np.array([[0,0,0],[1,1,1],[2,0,1],[3,1,3]]), columns=["x","y","score"])
adata.obs = t
bins_x = [0,2,3,4]
bins_y = [0,2]
# t.loc[(t["x"]>=bins_x[0])&(t["x"]<bins_x[1])&(t["y"]>=bins_y[0])&(t["y"]<=bins_y[1])]
t["test"] = 0
k = 0
            
for x_start, x_end in zip(bins_x[:-1], bins_x[1:]):
    for y_start, y_end in zip(bins_y[:-1], bins_y[1:]):
        df = t[
            (t["x"] >= x_start)
            & (t["x"] < x_end)
            & (t["y"] >= y_start)
            & (t["y"] < y_end)
        ]
        print(x_start, x_end)
        print(y_start, y_end)
        print(df.index)
